# DS-SF-30 | Codealong 03: Databases, Scrapping, and APIs; Part A - RDBMS Databases and `SQLite`

In [2]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### How to connect to a `SQLite` database

In [3]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'dataset-03-zillow.db'))

> ### How to use `pandas` to run SQL queries and get the results as a `DataFrame`

In [4]:
pd.io.sql.read_sql(
'''
SELECT *
    FROM properties
    LIMIT 10
;
''', con = db)

,ID,address,latitude,longitude,is_a_studio,...,size,size_unit,lot_size,lot_size_unit,built_in_year
0,15063471,"55 Vandewater St APT 9, San Francisco, CA",37805103,-122412856,0,...,550.0,sqft,NaN,None,1980.0
1,15063505,"740 Francisco St, San Francisco, CA",37804420,-122417389,0,...,1430.0,sqft,2435.0,sqft,1948.0
2,15063609,"819 Francisco St, San Francisco, CA",37803728,-122419055,0,...,2040.0,sqft,3920.0,sqft,1976.0
3,15064044,"199 Chestnut St APT 5, San Francisco, CA",37804392,-122406590,0,...,1060.0,sqft,NaN,None,1930.0
4,15064257,"111 Chestnut St APT 403, San Francisco, CA",37804240,-122405509,0,...,1299.0,sqft,NaN,None,1993.0
5,15064295,"111 Chestnut St APT 702, San Francisco, CA",37804240,-122405509,0,...,1033.0,sqft,NaN,None,1993.0
6,15064391,"1821 Grant Ave APT 101, San Francisco, CA",37803748,-122408531,0,...,1048.0,sqft,NaN,None,1975.0
7,15064536,"2300 Leavenworth St, San Francisco, CA",37802408,-122417537,0,...,2115.0,sqft,1271.0,sqft,1913.0
8,15064640,"1047-1049 Lombard St, San Francisco, CA",37801889,-122418704,1,...,4102.0,sqft,3049.0,sqft,1948.0
9,15064669,"1055 Lombard St # C, San Francisco, CA",37801873,-122418834,0,...,1182.0,sqft,NaN,None,1986.0


### `SELECT`

In [5]:
pd.io.sql.read_sql(
'''
SELECT ID, sale_price, sale_price_unit
    FROM transactions
;
''', con = db)

,ID,sale_price,sale_price_unit
0,15063471,710000.00,$
1,15063505,2.15,$M
2,15063609,5.60,$M
3,15064044,1.50,$M
4,15064257,970000.00,$
...,...,...,...
995,2124214951,390000.00,$
996,2126960082,860000.00,$
997,2128308939,830000.00,$
998,2131957929,835000.00,$


### `WHERE`

In [6]:
pd.io.sql.read_sql(
'''
SELECT ID, sale_price
    FROM transactions
        WHERE sale_price_unit = '$M'
;
''', con = db)

,ID,sale_price
0,15063505,2.15
1,15063609,5.60
2,15064044,1.50
3,15064536,2.83
4,15064640,4.05
...,...,...
570,2106229715,1.24
571,2108411930,1.73
572,2108546010,1.40
573,2121221385,1.43


> #### Here's let's convert the unit of `sale_price` from \$ to \$M

In [8]:
pd.io.sql.read_sql(
'''
SELECT ID, sale_price / 1000000 AS sale_price
    FROM transactions
        WHERE sale_price_unit = '$M'
;
''', con = db)

,ID,sale_price
0,15063505,0.000002
1,15063609,0.000006
2,15064044,0.000002
3,15064536,0.000003
4,15064640,0.000004
...,...,...
570,2106229715,0.000001
571,2108411930,0.000002
572,2108546010,0.000001
573,2121221385,0.000001


### `UNION`

> #### Let's combine the previous two queries into one.  `sale_price` is now in \$M accross the dataset

In [11]:
pd.io.sql.read_sql('''
SELECT ID, sale_price / 1000000
    FROM transactions
        WHERE sale_price_unit = '$'
UNION ALL
SELECT ID, sale_price AS sale_price
    FROM transactions
        WHERE sale_price_unit = '$M'
;
''', con = db)

,ID,sale_price / 1000000
0,15063471,0.710
1,15064257,0.970
2,15064295,0.940
3,15064391,0.835
4,15065032,0.800
...,...,...
995,2106229715,1.240
996,2108411930,1.730
997,2108546010,1.400
998,2121221385,1.430


### `WITH`

In [18]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price / 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$M')

SELECT *
    FROM normalized_sale_prices
;
''', con = db)

,ID,sale_price
0,15063471,0.710
1,15064257,0.970
2,15064295,0.940
3,15064391,0.835
4,15065032,0.800
...,...,...
995,2106229715,1.240
996,2108411930,1.730
997,2108546010,1.400
998,2121221385,1.430


### `JOIN`

In [21]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price / 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$M')

SELECT properties.id as id, properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price
    FROM properties
        JOIN normalized_sale_prices ON normalized_sale_prices.ID = properties.ID
;
''', con = db)

,id,beds,sale_price
0,15063471,1.0,0.710
1,15064257,2.0,0.970
2,15064295,2.0,0.940
3,15064391,1.0,0.835
4,15065032,1.0,0.800
...,...,...,...
995,2106229715,3.0,1.240
996,2108411930,2.0,1.730
997,2108546010,2.0,1.400
998,2121221385,NaN,1.430


### `WITH` (take 2)

In [22]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price / 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$M'),

beds_normalized_sale_prices AS
    (SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price
        FROM properties
            JOIN normalized_sale_prices
                ON normalized_sale_prices.ID = properties.ID)

SELECT * FROM beds_normalized_sale_prices
;
''', con = db)

,beds,sale_price
0,1.0,0.710
1,2.0,0.970
2,2.0,0.940
3,1.0,0.835
4,1.0,0.800
...,...,...
995,3.0,1.240
996,2.0,1.730
997,2.0,1.400
998,NaN,1.430


### `GROUP BY` and `AVG`

> ### Let's now compute the average sale price of these properties based on their number of bedrooms

In [27]:
pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price / 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$M'),

beds_normalized_sale_prices AS
    (SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price
        FROM properties
            JOIN normalized_sale_prices
                ON normalized_sale_prices.ID = properties.ID)

SELECT AVG(sale_price) AS mean_sale_price, beds
    FROM beds_normalized_sale_prices
        GROUP BY beds
;
''', con = db)

,mean_sale_price,beds
0,1.494858,NaN
1,0.820425,1.0
2,1.192398,2.0
3,1.404323,3.0
4,1.879330,4.0
5,2.867720,5.0
6,2.587222,6.0
7,5.543167,7.0
8,1.960000,8.0
9,1.766667,9.0


Note: `beds = 0` usually refers to studios but not always...

> ### Activity
> #### Compute the average sales price of studios

In [75]:
a = pd.io.sql.read_sql('''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price / 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$M'),

beds_normalized_sale_prices AS
    (SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price,
            properties.is_a_studio
        FROM properties
            JOIN normalized_sale_prices
                ON normalized_sale_prices.ID = properties.ID
            WHERE properties.is_a_studio = 1)

SELECT AVG(sale_price) AS mean_sale_price
    FROM beds_normalized_sale_prices
        GROUP BY beds
;
''', con = db)
print(a)

b = pd.io.sql.read_sql(
'''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price / 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$M'),

beds_normalized_sale_prices AS
    (SELECT properties.beds AS beds, normalized_sale_prices.sale_price AS sale_price,
            properties.is_a_studio    
        FROM properties
            JOIN normalized_sale_prices
                ON normalized_sale_prices.ID = properties.ID)

SELECT sale_price, beds, is_a_studio
    FROM beds_normalized_sale_prices
;
''', con = db)
b = b[b.is_a_studio == 1].sale_price.mean()
print('\nFrom Pandas: %.6f' % b)

   mean_sale_price
0         1.464034

From Pandas: 1.464034


> ### Activity
> #### Find the properties in the dataset that have different addresses but the same latitude/longitude.  Return the smallest set of pair of properties (`address_a`, `address_b`)

In [91]:
pd.io.sql.read_sql('''
SELECT 
    properties_a.id as id_a,
    properties_b.id as id_b,
    properties_a.address as address_a,
    properties_b.address as address_b
FROM properties properties_a
    JOIN properties properties_b
        ON properties_a.latitude = properties_b.latitude
        AND properties_a.longitude = properties_b.longitude
        AND properties_a.address <> properties_b.address
        AND properties_a.id < properties_b.id
;
''', con = db)

,id_a,id_b,address_a,address_b
0,15064257,15064295,"111 Chestnut St APT 403, San Francisco, CA","111 Chestnut St APT 702, San Francisco, CA"
1,15066001,15066036,"1070 Green St APT 201, San Francisco, CA","1070 Green St APT 1402, San Francisco, CA"
2,15067583,15067604,"946 Stockton St APT 6B, San Francisco, CA","946 Stockton St APT 9E, San Francisco, CA"
3,15067890,15067896,"1250 Jones St APT 503, San Francisco, CA","1250 Jones St APT 802, San Francisco, CA"
4,15068514,15068551,"1177 California St APT 702, San Francisco, CA","1177 California St APT 501, San Francisco, CA"
...,...,...,...,...
102,123174799,123174904,"1080 Sutter St APT 1003, San Francisco, CA","1080 Sutter St APT 1001, San Francisco, CA"
103,124397844,124397860,"2200 Market St APT 205, San Francisco, CA","2200 Market St APT 202, San Francisco, CA"
104,124850278,124862135,"8 Octavia St UNIT 401, San Francisco, CA","8 Octavia St, San Francisco, CA"
105,2100643449,2100786271,"2149 Lyon St APT 4, San Francisco, CA","2149 Lyon St APT 1, San Francisco, CA"
